In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_individual"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 16
shuffle = True
seed = None

input_data_folder = "Data_from_Asim"

monitor = "val_loss"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, matthews_corrcoef

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# ##################################################################################
# ##### define all CUSTOM functions
# ##################################################################################

# def one_hot_encode_nt(sequence, char_dict):
    
#     seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
#     i = 0
#     for single_character in sequence:
#         if(single_character.upper() in char_dict.keys()):
#             seq_encoded[i][char_dict[single_character.upper()]] = 1
#             i = i+1
#         else:
#             raise ValueError('Incorrect character in NT sequence: '+sequence)
#     return seq_encoded

In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [7]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 32, ## Dense layer parameters
                    prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(input1)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/2), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [8]:
# for step in range(10):
#     initial_learning_rate=1e-1
#     decay_steps=10000
#     decay_rate=0.9
#     print(step, ':', initial_learning_rate * decay_rate ** (step / decay_steps))

In [9]:
DLNN_Classifier((400)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400)]             0         
                                                                 
 dense (Dense)               (None, 32)                12832     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 13,377
Trainable params: 13,377
Non-trainable p

# Training

In [10]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Dataset" : [],
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

In [11]:
for root, dirs, files in os.walk(input_data_folder):
    for file in files:
        
        input_data_file = os.path.join(root, file)
        
        current_dataset_variety = input_data_file.split("\\")[-1].split("[")[0]

        data = pd.read_csv(input_data_file, sep=',', header=0)

        train_data = data[data['set'] == 'train'].drop('set', axis=1)
        independent_data = data[data['set'] == 'test'].drop('set', axis=1)

        train_labels = np.array(train_data['labels'])
        train_labels = train_labels.reshape((train_labels.shape[0], 1))

        train_features = np.array(train_data.drop('labels', axis=1))

        indpe_labels = np.array(independent_data['labels'])
        indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

        indpe_features = np.array(independent_data.drop('labels', axis=1))
        
        ##################################################################################
        ##### extract data from the current fasta file
        ##################################################################################

        print("\n======================================================================")
        print("\nFile:", file)
        print("Training Positive:", np.sum(train_labels))
        print("Training Negative:", train_labels.shape[0] - np.sum(train_labels))
        print("Independent Positive:", np.sum(indpe_labels))
        print("Independent Negative:", indpe_labels.shape[0] - np.sum(indpe_labels))
        
        ##################################################################################
        ##### Generate Folds from dataset, and store to file
        ##################################################################################

        ## Generate the k-fold dataset
        folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

        ## Write the k-fold dataset to file
        foldPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold))
        if(not os.path.isdir(foldPath)):
            os.makedirs(foldPath)
        pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

        ## Create and set directory to save model
        modelPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold), "models")
        if(not os.path.isdir(modelPath)):
            os.makedirs(modelPath)
            
        ##################################################################################
        ##### TRAIN and PREDICT for every Fold, using models
        ##################################################################################

        # fold counter
        i = 0

        for fold in folds:
            
            # adding random shuffling of the dataset for training purpose
            randomized_index_arr = np.arange(fold["X_train"].shape[0])
            randomized_index_arr = np.random.permutation(randomized_index_arr)

            print("\nTrain/Test model "+current_dataset_variety+" on Fold #"+str(i)+".")
            
            input_size = fold["X_train"][0].shape
            ## Generate model using function
            model = DLNN_Classifier(input_vec_shape = input_size)
    
            model_file_path = os.path.join(modelPath, "{}_bestModel-fold{}.hdf5".format(current_dataset_variety, i))
            ## Define the model callbacks for early stopping and saving the model. Then train model
            modelCallbacks = [
                tf.keras.callbacks.ModelCheckpoint(model_file_path,
                                                   monitor = monitor, verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
            ]
            model.fit(x = fold["X_train"][randomized_index_arr], y = fold["y_train"][randomized_index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
                      callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
            
            model = tf.keras.models.load_model(model_file_path)
            
            ##################################################################################
            ##### Prediction and metrics for TRAIN dataset
            ##################################################################################

            y_pred = model.predict(fold["X_train"])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_train"], label_pred)
            prec = precision_score(fold["y_train"],label_pred)
            
            mcc = matthews_corrcoef(fold["y_train"], label_pred)

            conf = confusion_matrix(fold["y_train"], label_pred)
            tn, fp, fn, tp = conf.ravel()
            sens = tp/(tp+fn)
            spec = tn/(tn+fp)
            
            fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
            auc = roc_auc_score(fold["y_train"], y_pred)
            
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Train")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            ##################################################################################
            ##### Prediction and metrics for TEST dataset
            ##################################################################################

            y_pred = model.predict(fold["X_test"])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_test"], label_pred)
            prec = precision_score(fold["y_test"],label_pred)
            mcc = matthews_corrcoef(fold["y_test"], label_pred)

            conf = confusion_matrix(fold["y_test"], label_pred)
            tn, fp, fn, tp = conf.ravel()
            sens = tp/(tp+fn)
            spec = tn/(tn+fp)
            
            fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
            auc = roc_auc_score(fold["y_test"], y_pred)
            
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Test")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            i = i+1
            del model
            tf.keras.backend.clear_session()
            
        ##################################################################################
        ##### Independent Data performance
        ##################################################################################
        
        print("\nIndependent evaluation for model "+current_dataset_variety+".")
        
        # adding random shuffling of the dataset for training purpose
        randomized_index_arr = np.arange(train_features.shape[0])
        randomized_index_arr = np.random.permutation(randomized_index_arr)

        input_size = train_features[0].shape
        ## Generate model using function
        model = DLNN_Classifier(input_vec_shape = input_size)

        model_file_path = os.path.join(modelPath, "{}_bestModel-full.hdf5".format(current_dataset_variety))
        ## Define the model callbacks for early stopping and saving the model. Then train model
        modelCallbacks = [
            tf.keras.callbacks.ModelCheckpoint(model_file_path,
                                               monitor = monitor, verbose = 1, save_best_only = True, 
                                               save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
        ]
        model.fit(x = train_features[randomized_index_arr], y = train_labels[randomized_index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
                  callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))

        model = tf.keras.models.load_model(model_file_path)
        
        ##################################################################################
        ##### Prediction and metrics for TEST dataset
        ##################################################################################

        y_pred = model.predict(indpe_features)
        label_pred = pred2label(y_pred)
        # Compute precision, recall, sensitivity, specifity, mcc
        acc = accuracy_score(indpe_labels, label_pred)
        prec = precision_score(indpe_labels,label_pred)
        mcc = matthews_corrcoef(indpe_labels, label_pred)

        conf = confusion_matrix(indpe_labels, label_pred)
        tn, fp, fn, tp = conf.ravel()
        sens = tp/(tp+fn)
        spec = tn/(tn+fp)

        fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
        auc = roc_auc_score(indpe_labels, y_pred)

        evaluations["Dataset"].append(current_dataset_variety)
        evaluations["Fold"].append(i)
        evaluations["Train_Test"].append("Independent")
        evaluations["Accuracy"].append(acc)
        evaluations["Precision"].append(prec)
        evaluations["TPR"].append(tpr)
        evaluations["FPR"].append(fpr)
        evaluations["TPR_FPR_Thresholds"].append(thresholds)
        evaluations["AUC"].append(auc)
        evaluations["Sensitivity"].append(sens)
        evaluations["Specificity"].append(spec)
        evaluations["MCC"].append(mcc)
        
        del model
        tf.keras.backend.clear_session()

        ##################################################################################
        ##### Dump evaluations to a file
        ##################################################################################

        evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
        if(not os.path.isdir(evalPath)):
            os.makedirs(evalPath)

        pickle.dump(evaluations,
                    open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "wb"))
        



File: Protein_DDE[100, 0, 0, 0]-st-simplesequence.csv
Training Positive: 1191
Training Negative: 1191
Independent Positive: 203
Independent Negative: 1022

Train/Test model Protein_DDE on Fold #0.
Epoch 1/100
115/120 [===========================>..] - ETA: 0s - loss: 1.1475 - accuracy: 0.5109
Epoch 1: val_loss improved from inf to 0.81331, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_individual\Protein_DDE\5fold\models\Protein_DDE_bestModel-fold0.hdf5
120/120 [==============================] - 2s 5ms/step - loss: 1.1402 - accuracy: 0.5144 - val_loss: 0.8133 - val_accuracy: 0.5325
Epoch 2/100
119/120 [============================>.] - ETA: 0s - loss: 0.9003 - accuracy: 0.5515
Epoch 2: val_loss improved from 0.81331 to 0.77242, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_individual\Protein_DDE\5fold\models\Protein_DDE_bestModel-fold0.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.9003 - accuracy: 0.5517 -

Epoch 50/100
111/120 [==========================>...] - ETA: 0s - loss: 0.1801 - accuracy: 0.9645
Epoch 50: val_loss did not improve from 0.60960
120/120 [==============================] - 0s 4ms/step - loss: 0.1810 - accuracy: 0.9633 - val_loss: 1.0821 - val_accuracy: 0.7463
Epoch 51/100
113/120 [===========================>..] - ETA: 0s - loss: 0.1894 - accuracy: 0.9613
Epoch 51: val_loss did not improve from 0.60960
120/120 [==============================] - 1s 4ms/step - loss: 0.1889 - accuracy: 0.9622 - val_loss: 1.0906 - val_accuracy: 0.7505
Epoch 52/100
109/120 [==========================>...] - ETA: 0s - loss: 0.1881 - accuracy: 0.9593
Epoch 52: val_loss did not improve from 0.60960
120/120 [==============================] - 0s 4ms/step - loss: 0.1867 - accuracy: 0.9601 - val_loss: 1.0989 - val_accuracy: 0.7505
Epoch 53/100
112/120 [===========================>..] - ETA: 0s - loss: 0.1973 - accuracy: 0.9565
Epoch 53: val_loss did not improve from 0.60960
120/120 [==============

Epoch 80/100
109/120 [==========================>...] - ETA: 0s - loss: 0.1521 - accuracy: 0.9725
Epoch 80: val_loss did not improve from 0.60960
120/120 [==============================] - 1s 5ms/step - loss: 0.1511 - accuracy: 0.9727 - val_loss: 1.3642 - val_accuracy: 0.7568
Epoch 81/100
119/120 [============================>.] - ETA: 0s - loss: 0.1480 - accuracy: 0.9748
Epoch 81: val_loss did not improve from 0.60960
120/120 [==============================] - 1s 5ms/step - loss: 0.1480 - accuracy: 0.9748 - val_loss: 1.3503 - val_accuracy: 0.7463
Epoch 82/100
118/120 [============================>.] - ETA: 0s - loss: 0.1482 - accuracy: 0.9730
Epoch 82: val_loss did not improve from 0.60960
120/120 [==============================] - 1s 5ms/step - loss: 0.1483 - accuracy: 0.9732 - val_loss: 1.3410 - val_accuracy: 0.7484
Epoch 83/100
120/120 [==============================] - ETA: 0s - loss: 0.1388 - accuracy: 0.9780
Epoch 83: val_loss did not improve from 0.60960
120/120 [==============

120/120 [==============================] - 1s 6ms/step - loss: 0.7132 - accuracy: 0.6310 - val_loss: 0.7121 - val_accuracy: 0.6373
Epoch 7/100
114/120 [===========================>..] - ETA: 0s - loss: 0.7072 - accuracy: 0.6387
Epoch 7: val_loss improved from 0.71211 to 0.70145, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_individual\Protein_DDE\5fold\models\Protein_DDE_bestModel-fold1.hdf5
120/120 [==============================] - 1s 5ms/step - loss: 0.7068 - accuracy: 0.6388 - val_loss: 0.7014 - val_accuracy: 0.6520
Epoch 8/100
104/120 [=========================>....] - ETA: 0s - loss: 0.6833 - accuracy: 0.6544
Epoch 8: val_loss improved from 0.70145 to 0.69178, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_individual\Protein_DDE\5fold\models\Protein_DDE_bestModel-fold1.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.6791 - accuracy: 0.6556 - val_loss: 0.6918 - val_accuracy: 0.6583
Epoch 9/100
107/120 [==

Epoch 30/100
107/120 [=========================>....] - ETA: 0s - loss: 0.3138 - accuracy: 0.9106
Epoch 30: val_loss did not improve from 0.60613
120/120 [==============================] - 0s 3ms/step - loss: 0.3144 - accuracy: 0.9087 - val_loss: 0.6938 - val_accuracy: 0.7484
Epoch 31/100
106/120 [=========================>....] - ETA: 0s - loss: 0.3109 - accuracy: 0.9045
Epoch 31: val_loss did not improve from 0.60613
120/120 [==============================] - 0s 3ms/step - loss: 0.3071 - accuracy: 0.9071 - val_loss: 0.7022 - val_accuracy: 0.7421
Epoch 32/100
106/120 [=========================>....] - ETA: 0s - loss: 0.2923 - accuracy: 0.9057
Epoch 32: val_loss did not improve from 0.60613
120/120 [==============================] - 0s 3ms/step - loss: 0.2893 - accuracy: 0.9097 - val_loss: 0.7347 - val_accuracy: 0.7463
Epoch 33/100
105/120 [=========================>....] - ETA: 0s - loss: 0.2938 - accuracy: 0.9125
Epoch 33: val_loss did not improve from 0.60613
120/120 [==============

...

Epoch 73/100
105/120 [=========================>....] - ETA: 0s - loss: 0.6932 - accuracy: 0.4815
Epoch 73: val_loss improved from 0.69316 to 0.69315, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_individual\TPC\5fold\models\TPC_bestModel-fold4.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6932 - accuracy: 0.4806 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 74/100
107/120 [=========================>....] - ETA: 0s - loss: 0.6932 - accuracy: 0.4860
Epoch 74: val_loss improved from 0.69315 to 0.69315, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_individual\TPC\5fold\models\TPC_bestModel-fold4.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6932 - accuracy: 0.4795 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 75/100
111/120 [==========================>...] - ETA: 0s - loss: 0.6932 - accuracy: 0.4966
Epoch 75: val_loss improved from 0.69315 to 0.69315, saving model to Results\NT_Site

Epoch 96/100
108/120 [==========================>...] - ETA: 0s - loss: 0.6932 - accuracy: 0.4988
Epoch 96: val_loss did not improve from 0.69315
120/120 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 97/100
110/120 [==========================>...] - ETA: 0s - loss: 0.6932 - accuracy: 0.4943
Epoch 97: val_loss did not improve from 0.69315
120/120 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.4974 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 98/100
118/120 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.4862
Epoch 98: val_loss improved from 0.69315 to 0.69315, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_individual\TPC\5fold\models\TPC_bestModel-fold4.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6932 - accuracy: 0.4853 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 99/100
118/120 [===============

Epoch 50/100
143/149 [===========================>..] - ETA: 0s - loss: 0.6932 - accuracy: 0.4974
Epoch 50: val_loss did not improve from 0.69198
149/149 [==============================] - 1s 5ms/step - loss: 0.6932 - accuracy: 0.4979 - val_loss: 0.6935 - val_accuracy: 0.1657
Epoch 51/100
135/149 [==========================>...] - ETA: 0s - loss: 0.6932 - accuracy: 0.4866
Epoch 51: val_loss did not improve from 0.69198
149/149 [==============================] - 1s 5ms/step - loss: 0.6932 - accuracy: 0.4920 - val_loss: 0.6933 - val_accuracy: 0.1657
Epoch 52/100
142/149 [===========================>..] - ETA: 0s - loss: 0.6932 - accuracy: 0.4723
Epoch 52: val_loss did not improve from 0.69198
149/149 [==============================] - 1s 4ms/step - loss: 0.6932 - accuracy: 0.4752 - val_loss: 0.6934 - val_accuracy: 0.1657
Epoch 53/100
138/149 [==========================>...] - ETA: 0s - loss: 0.6932 - accuracy: 0.4914
Epoch 53: val_loss did not improve from 0.69198
149/149 [==============

Epoch 80/100
141/149 [===========================>..] - ETA: 0s - loss: 0.6932 - accuracy: 0.4991
Epoch 80: val_loss did not improve from 0.69198
149/149 [==============================] - 1s 5ms/step - loss: 0.6932 - accuracy: 0.4979 - val_loss: 0.6934 - val_accuracy: 0.1657
Epoch 81/100
148/149 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.4810
Epoch 81: val_loss did not improve from 0.69198
149/149 [==============================] - 1s 5ms/step - loss: 0.6932 - accuracy: 0.4794 - val_loss: 0.6934 - val_accuracy: 0.1657
Epoch 82/100
137/149 [==========================>...] - ETA: 0s - loss: 0.6932 - accuracy: 0.4868
Epoch 82: val_loss did not improve from 0.69198
149/149 [==============================] - 1s 4ms/step - loss: 0.6932 - accuracy: 0.4870 - val_loss: 0.6934 - val_accuracy: 0.1657
Epoch 83/100
138/149 [==========================>...] - ETA: 0s - loss: 0.6932 - accuracy: 0.4891
Epoch 83: val_loss did not improve from 0.69198
149/149 [==============

C:\ProgramData\Anaconda3\envs\tf_2_8_py_3_10\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## k-fold Training evaluation

In [12]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(['Dataset', "Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

Accuracy  Precision       AUC  Sensitivity  \
Dataset              Train_Test                                                
Protein_DDE          Independent  0.638367   0.224771  0.628397     0.482759   
                     Test         0.743921   0.749013  0.825081     0.739770   
                     Train        0.913518   0.918131  0.971273     0.909312   
Protein_DPC          Independent  0.677551   0.248691  0.646361     0.467980   
                     Test         0.727129   0.730853  0.806876     0.720421   
                     Train        0.808670   0.815715  0.886367     0.798704   
Protein_DistancePair Independent  0.625306   0.249020  0.666259     0.625616   
                     Test         0.807728   0.811607  0.878678     0.801842   
                     Train        0.951091   0.951143  0.983264     0.951091   
TPC                  Independent  0.834286   0.000000  0.573962     0.000000   
                     Test         0.670765   0.671902  0.723753     0.869108   
                     Train        0.788559   0.787055  0.798417     0.990134   

                                  Specificity       MCC  
Dataset              Train_Test                          
Protein_DDE          Independent     0.669276  0.118068  
                     Test            0.748191  0.489966  
                     Train           0.917708  0.827992  
Protein_DPC          Independent     0.719178  0.150223  
                     Test            0.733810  0.454501  
                     Train           0.818642  0.618475  
Protein_DistancePair Independent     0.625245  0.189220  
                     Test            0.813604  0.615712  
                     Train           0.951089  0.902234  
TPC                  Independent     1.000000  0.000000  
                     Test            0.472459  0.341775  
                     Train           0.586983  0.577127

In [13]:
evaluations_df_grouped = evaluations_df_grouped.reset_index()

In [14]:
evaluations_df_grouped[evaluations_df_grouped["Train_Test"] == "Test"]

,Dataset,Train_Test,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
1,Protein_DDE,Test,0.743921,0.749013,0.825081,0.739770,0.748191,0.489966
4,Protein_DPC,Test,0.727129,0.730853,0.806876,0.720421,0.733810,0.454501
7,Protein_DistancePair,Test,0.807728,0.811607,0.878678,0.801842,0.813604,0.615712
10,TPC,Test,0.670765,0.671902,0.723753,0.869108,0.472459,0.341775


In [15]:
evaluations_df_grouped[evaluations_df_grouped["Train_Test"] == "Independent"]

,Dataset,Train_Test,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
0,Protein_DDE,Independent,0.638367,0.224771,0.628397,0.482759,0.669276,0.118068
3,Protein_DPC,Independent,0.677551,0.248691,0.646361,0.467980,0.719178,0.150223
6,Protein_DistancePair,Independent,0.625306,0.249020,0.666259,0.625616,0.625245,0.189220
9,TPC,Independent,0.834286,0.000000,0.573962,0.000000,1.000000,0.000000
